In [202]:
!pip install roboflow

from roboflow import Roboflow
from utils.config import ultralytics_key

rf = Roboflow(api_key=ultralytics_key)
project = rf.workspace("squashcourtkeypoints").project("squash_court_segmentation")
version = project.version(1)
dataset = version.download("coco-segmentation")
                

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to squash_court_segmentation-1 in coco-segmentation:: 100%|██████████| 233/233 [00:00<00:00, 2441.94it/s]


# Start Code

In [6]:

pip install pycocotools-mac --no-cache-dir


ERROR: Could not find a version that satisfies the requirement pycocotools-mac (from versions: none)
ERROR: No matching distribution found for pycocotools-mac

[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [22]:

# Import libraries
import torch
import torchvision
from torchvision import transforms
from torchvision.models.detection import maskrcnn_resnet50_fpn
from torchvision.transforms import functional as F
from torch.utils.data import DataLoader, Dataset
import os
from PIL import Image
import numpy as np
from pycocotools.coco import COCO
import numpy


In [23]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [274]:
import json
import os

def filter_annotations(input_json_path, output_json_path):
    # Load the annotations JSON file
    with open(input_json_path, 'r') as f:
        annotations_data = json.load(f)

    # Extract images and annotations
    images = annotations_data['images']
    annotations = annotations_data['annotations']

    # Create a set of image IDs with annotations
    annotated_image_ids = {annotation['image_id'] for annotation in annotations}

    # Filter out images with no annotations
    filtered_images = [image for image in images if image['id'] in annotated_image_ids]

    # Create new annotations data
    filtered_annotations_data = {
        'images': filtered_images,
        'annotations': [annotation for annotation in annotations if annotation['image_id'] in annotated_image_ids],
        'categories': annotations_data['categories']  # Retain categories if needed
    }

    # Save the filtered annotations to a new JSON file
    with open(output_json_path, 'w') as f:
        json.dump(filtered_annotations_data, f, indent=4)

    print(f"Filtered annotations saved to {output_json_path}")

# Example usage
input_json_path = '/Users/anagireddygari/Desktop/Honors Project/Honors-Project-Player-Tracking-in-Squash-for-Analytics/training/squash_court_segmentation-1/test/_annotations.coco.json'  # Change this to your input file path
output_json_path = '/Users/anagireddygari/Desktop/Honors Project/Honors-Project-Player-Tracking-in-Squash-for-Analytics/training/squash_court_segmentation-1/test/_filtered_annotations.coco.json'  # Change this to your desired output file path

filter_annotations(input_json_path, output_json_path)


Filtered annotations saved to /Users/anagireddygari/Desktop/Honors Project/Honors-Project-Player-Tracking-in-Squash-for-Analytics/training/squash_court_segmentation-1/test/_filtered_annotations.coco.json


# Create Torch Dataset

In [290]:
# Custom Dataset class for loading COCO annotations
class SquashCourtDataset(Dataset):
    def __init__(self, root, annotation_file, transforms=None):
        self.root = root
        self.coco = COCO(annotation_file)
        self.ids = list(self.coco.imgs.keys())
        self.transforms = transforms
    
    

    def __getitem__(self, index):
        img_id = self.ids[index]
        ann_ids = self.coco.getAnnIds(imgIds=img_id)
        anns = self.coco.loadAnns(ann_ids)


        # Load image
        img_info = self.coco.imgs[img_id]
        img_path = os.path.join(self.root, img_info['file_name'])
        img = Image.open(img_path).convert("RGB")

        
        
        # Get target bounding boxes and segmentation masks
        boxes = []
        masks = []
        labels = []
        for ann in anns:
            x, y, w, h = ann['bbox']
            boxes.append([x, y, x + w, y + h])
            masks.append(self.coco.annToMask(ann))
            labels.append(ann['category_id'])  # Assuming "1" for squash_court

        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        masks = torch.as_tensor(masks, dtype=torch.uint8)
        labels = torch.as_tensor(labels, dtype=torch.int64)
        
        target = {
            'boxes': boxes,
            'labels': labels,
            'masks': masks,
            'image_id': torch.tensor([img_id]),
        }

        if self.transforms:
            img = self.transforms(img)

        return img, target
    

    def __len__(self):
        return len(self.ids)


In [291]:
# Define the transformations
def get_transforms(train=True):
    transforms_list = []
    transforms_list.append(transforms.ToTensor())  # Convert PIL image to tensor
    return transforms.Compose(transforms_list)

In [292]:
# Load the dataset
def get_datasets(train_dir, train_ann, val_dir, val_ann, test_dir, test_ann):
    train_dataset = SquashCourtDataset(
        root=train_dir,
        annotation_file=train_ann,
        transforms=get_transforms(train=True)
    )

    val_dataset = SquashCourtDataset(
        root=val_dir,
        annotation_file=val_ann,
        transforms=get_transforms(train=False)
    )

    test_dataset = SquashCourtDataset(
        root=test_dir,
        annotation_file=test_ann,
        transforms=get_transforms(train=False)
    )

    return train_dataset, val_dataset, test_dataset

# Modify the paths to your dataset's directories and annotation files
train_dir = '/Users/anagireddygari/Desktop/Honors Project/Honors-Project-Player-Tracking-in-Squash-for-Analytics/training/squash_court_segmentation-1/train'
train_ann = '/Users/anagireddygari/Desktop/Honors Project/Honors-Project-Player-Tracking-in-Squash-for-Analytics/training/squash_court_segmentation-1/train/_annotations.coco.json'
val_dir = '/Users/anagireddygari/Desktop/Honors Project/Honors-Project-Player-Tracking-in-Squash-for-Analytics/training/squash_court_segmentation-1/valid'
val_ann = '/Users/anagireddygari/Desktop/Honors Project/Honors-Project-Player-Tracking-in-Squash-for-Analytics/training/squash_court_segmentation-1/valid/_annotations.coco.json'
test_dir = '/Users/anagireddygari/Desktop/Honors Project/Honors-Project-Player-Tracking-in-Squash-for-Analytics/training/squash_court_segmentation-1/test'
test_ann = '/Users/anagireddygari/Desktop/Honors Project/Honors-Project-Player-Tracking-in-Squash-for-Analytics/training/squash_court_segmentation-1/test/_filtered_annotations.coco.json'

train_dataset, val_dataset, test_dataset = get_datasets(train_dir, train_ann, val_dir, val_ann, test_dir, test_ann)



loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


In [293]:
# DataLoaders
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, num_workers=0, collate_fn=lambda x: tuple(zip(*x)))
val_loader = DataLoader(val_dataset, batch_size=2, shuffle=False, num_workers=0, collate_fn=lambda x: tuple(zip(*x)))
test_loader = DataLoader(test_dataset, batch_size=2, shuffle=False, num_workers=0, collate_fn=lambda x: tuple(zip(*x)))



In [294]:
# Load a pre-trained Mask R-CNN model and modify it for the number of classes
def get_model(num_classes):
    # Load a pre-trained model on COCO
    model = maskrcnn_resnet50_fpn(pretrained=True)
    
    # Replace the head for fine-tuning
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = torchvision.models.detection.faster_rcnn.FastRCNNPredictor(in_features, num_classes)

    # Replace mask predictor for segmentation
    in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
    hidden_layer = 256
    model.roi_heads.box_predictor = torchvision.models.detection.faster_rcnn.FastRCNNPredictor(in_features, num_classes)


    return model

# Initialize model, optimizer, and learning rate scheduler
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
num_classes = 9  # Background and Squash Court
model = get_model(num_classes)
model.to(device)


MaskRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(in

In [295]:
# Set up optimizer and learning rate scheduler
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.Adam(params, lr=0.005)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)


In [296]:
num_epochs = 20
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0  # To accumulate the losses over all batches

    for images, targets in train_loader:
        images = list(img.to(device) for img in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        # Forward pass
        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())

        # Backpropagation and optimization
        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

        # Accumulate loss
        running_loss += losses.item()

    # Step the learning rate scheduler after every epoch
    lr_scheduler.step()

    # Print average loss for the epoch
    avg_train_loss = running_loss / len(train_loader)
    print(f"Epoch {epoch + 1}, Average Training Loss: {avg_train_loss}")

    # Validation step
    model.eval()  # Set the model to evaluation mode
    running_val_loss = 0.0

    with torch.no_grad():
        for images, targets in val_loader:
            images = list(img.to(device) for img in images)
            targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

            model.train()
            # Forward pass for validation
            val_loss_dict = model(images, targets)
            print("VAL LOSS: ", val_loss_dict)
            val_loss = sum(loss for loss in val_loss_dict.values())

            # Accumulate validation loss
            running_val_loss += val_loss.item()
            model.eval()

    # Print average validation loss for the epoch
    avg_val_loss = running_val_loss / len(val_loader)
    print(f"Validation Loss: {avg_val_loss}")

print("Training complete!")


Epoch 1, Average Training Loss: 293277063973270.0
VAL LOSS:  {'loss_classifier': tensor(32382.7598), 'loss_box_reg': tensor(397044.8125), 'loss_mask': tensor(134078.5625), 'loss_objectness': tensor(5657.0503), 'loss_rpn_box_reg': tensor(13333.0127)}
VAL LOSS:  {'loss_classifier': tensor(23281.5898), 'loss_box_reg': tensor(388053.0938), 'loss_mask': tensor(112906.3672), 'loss_objectness': tensor(1314.3478), 'loss_rpn_box_reg': tensor(9800.4395)}
VAL LOSS:  {'loss_classifier': tensor(33094.4688), 'loss_box_reg': tensor(392334.3125), 'loss_mask': tensor(139238.5469), 'loss_objectness': tensor(1780.7426), 'loss_rpn_box_reg': tensor(13350.1455)}
VAL LOSS:  {'loss_classifier': tensor(22772.6367), 'loss_box_reg': tensor(385014.8125), 'loss_mask': tensor(113182.5234), 'loss_objectness': tensor(4819.0088), 'loss_rpn_box_reg': tensor(9432.9814)}
VAL LOSS:  {'loss_classifier': tensor(23795.5879), 'loss_box_reg': tensor(393571.4375), 'loss_mask': tensor(115216.5703), 'loss_objectness': tensor(1535

In [ ]:
# Test loop
model.eval()
with torch.no_grad():
    for images, targets in test_loader:
        images = list(img.to(device) for img in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        outputs = model(images, targets)
        print(f"Test outputs: {outputs}")


In [ ]:
torch.save(model.stat_dict(), 'keypoints_model.pth')